# Python 総合課題実習

授業で学んだデータインポート・外れ値処理・可視化・回帰分析を統合的に活用した実践分析

## 実装内容
1. データインポート・連結
2. 外れ値処理
3. 基本統計量
4. ヒストグラム・散布図作成
5. クロス集計・相関分析
6. ダミー変数
7. t検定
8. 回帰分析

In [ ]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# 日本語フォントの設定
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'Meiryo']
plt.rcParams['axes.unicode_minus'] = False

print("✓ ライブラリの読み込み完了")

---

## 1. データインポート・連結

### 📁 ファイルパスを指定してください

**1つのファイルの場合:**
```python
ファイルリスト = ['data.csv']
```

**複数のファイルを連結する場合:**
```python
ファイルリスト = [
    'data1.csv',
    'data2.csv',
    'data3.csv'
]
```

In [ ]:
# ========================================
# ここにファイルパスを入力してください
# ========================================

ファイルリスト = [
    'あなたのファイル.csv'  # ← ここを変更
]

# ========================================
# 以下は自動実行されます
# ========================================

データフレームリスト = []

for i, ファイルパス in enumerate(ファイルリスト):
    print(f"[{i+1}/{len(ファイルリスト)}] '{ファイルパス}' を読み込み中...")
    
    try:
        if ファイルパス.endswith('.csv'):
            df_temp = pd.read_csv(ファイルパス, encoding='utf-8-sig')
        elif ファイルパス.endswith('.txt'):
            try:
                df_temp = pd.read_csv(ファイルパス, sep='\t', encoding='utf-8-sig')
            except:
                df_temp = pd.read_csv(ファイルパス, encoding='utf-8-sig')
        else:
            df_temp = pd.read_csv(ファイルパス, encoding='utf-8-sig')
        
        print(f"  ✓ 成功: {df_temp.shape[0]}行 × {df_temp.shape[1]}列")
        データフレームリスト.append(df_temp)
        
    except Exception as e:
        print(f"  ✗ エラー: {e}")

# データ連結
if len(データフレームリスト) == 0:
    raise ValueError("ファイルを読み込めませんでした。ファイルパスを確認してください。")
elif len(データフレームリスト) == 1:
    df = データフレームリスト[0]
    print(f"\n✓ データ読み込み完了")
else:
    df = pd.concat(データフレームリスト, ignore_index=True)
    print(f"\n✓ {len(データフレームリスト)}個のファイルを連結しました")

print(f"最終データ: {df.shape[0]}行 × {df.shape[1]}列\n")
print("データの先頭5行:")
df.head()

---

## 2. 外れ値処理

IQR法による外れ値の検出と削除

In [ ]:
# 外れ値処理する列を指定
対象列 = 'price'  # ← 処理したい列名を入力

# IQR法
Q1 = df[対象列].quantile(0.25)
Q3 = df[対象列].quantile(0.75)
IQR = Q3 - Q1
下限 = Q1 - 1.5 * IQR
上限 = Q3 + 1.5 * IQR

外れ値 = df[(df[対象列] < 下限) | (df[対象列] > 上限)]

print(f"【外れ値検出結果】")
print(f"Q1 (25%): {Q1:.2f}")
print(f"Q3 (75%): {Q3:.2f}")
print(f"IQR: {IQR:.2f}")
print(f"下限: {下限:.2f}")
print(f"上限: {上限:.2f}")
print(f"外れ値の数: {len(外れ値)}個")

if len(外れ値) > 0:
    print(f"\n外れ値の例:")
    display(外れ値[[対象列]].head())
    
    # 外れ値を削除
    df = df[(df[対象列] >= 下限) & (df[対象列] <= 上限)].copy()
    print(f"\n✓ 外れ値を削除しました。残り: {len(df)}行")
else:
    print("\n外れ値はありませんでした。")

---

## 3. 基本統計量

In [ ]:
print("【基本統計量】")
df.describe()

---

## 4. ヒストグラム・散布図作成

In [ ]:
# ヒストグラム
数値df = df.select_dtypes(include=[np.number])

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, col in enumerate(数値df.columns[:6]):
    数値df[col].hist(bins=30, ax=axes[i], edgecolor='black')
    axes[i].set_title(f'{col} の分布', fontsize=12)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('度数')

for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.savefig('ヒストグラム.png', dpi=150, bbox_inches='tight')
print("✓ 'ヒストグラム.png' を保存しました")
plt.show()

In [ ]:
# 散布図行列
print("散布図を作成中...")
sns.pairplot(df.select_dtypes(include=[np.number]).iloc[:, :4])
plt.savefig('散布図.png', dpi=150, bbox_inches='tight')
print("✓ '散布図.png' を保存しました")
plt.show()

---

## 5. クロス集計・相関分析

In [ ]:
# 相関分析
print("【相関行列】")
相関行列 = 数値df.corr()
display(相関行列)

# 相関ヒートマップ
plt.figure(figsize=(10, 8))
sns.heatmap(相関行列, annot=True, cmap='coolwarm', center=0, 
            fmt='.2f', square=True, linewidths=1)
plt.title('相関行列ヒートマップ', fontsize=16, pad=20)
plt.tight_layout()
plt.savefig('相関分析.png', dpi=150, bbox_inches='tight')
print("✓ '相関分析.png' を保存しました")
plt.show()

In [ ]:
# クロス集計
カテゴリ列 = df.select_dtypes(exclude=[np.number]).columns.tolist()

if len(カテゴリ列) >= 2:
    print(f"【クロス集計】 ({カテゴリ列[0]} × {カテゴリ列[1]})")
    クロス表 = pd.crosstab(df[カテゴリ列[0]], df[カテゴリ列[1]])
    display(クロス表)

---

## 6. ダミー変数

カテゴリ変数を数値化

In [ ]:
if カテゴリ列:
    print(f"カテゴリ変数を数値化: {', '.join(カテゴリ列)}")
    df_ダミー = pd.get_dummies(df, columns=カテゴリ列, drop_first=True)
    print(f"✓ ダミー変数化完了。列数: {df.shape[1]} → {df_ダミー.shape[1]}")
else:
    df_ダミー = df.copy()
    print("カテゴリ変数がありません")

df_ダミー.head()

---

## 7. t検定

2群間の平均値の差を検定

In [ ]:
if カテゴリ列 and len(数値df.columns) > 0:
    カテゴリ = カテゴリ列[0]
    グループ = df[カテゴリ].unique()[:2]
    
    if len(グループ) >= 2:
        数値変数 = 数値df.columns[0]
        
        グループ1 = df[df[カテゴリ] == グループ[0]][数値変数].dropna()
        グループ2 = df[df[カテゴリ] == グループ[1]][数値変数].dropna()
        
        t統計量, p値 = stats.ttest_ind(グループ1, グループ2)
        
        print(f"【t検定結果】")
        print(f"変数: {数値変数}")
        print(f"グループ1 ({グループ[0]}): 平均={グループ1.mean():.2f}, n={len(グループ1)}")
        print(f"グループ2 ({グループ[1]}): 平均={グループ2.mean():.2f}, n={len(グループ2)}")
        print(f"\nt統計量: {t統計量:.4f}")
        print(f"p値: {p値:.4f}")
        
        if p値 < 0.05:
            print("→ 有意差あり (p < 0.05)")
        else:
            print("→ 有意差なし (p >= 0.05)")

---

## 8. 回帰分析

### 📊 変数を指定してください

In [ ]:
# ========================================
# ここで変数を指定してください
# ========================================

目的変数 = 'price'  # ← 予測したい変数
説明変数 = ['number_of_reviews', 'availability_365']  # ← 予測に使う変数

# ========================================
# 以下は自動実行されます
# ========================================

分析df = df_ダミー[[目的変数] + 説明変数].dropna()

X = 分析df[説明変数]
y = 分析df[目的変数]
X = sm.add_constant(X)

モデル = sm.OLS(y, X).fit()

print("="*70)
print("【回帰分析結果】")
print("="*70)
print(モデル.summary())

with open('回帰分析結果.txt', 'w', encoding='utf-8') as f:
    f.write(モデル.summary().as_text())

print("\n✓ '回帰分析結果.txt' に保存しました")

---

## ✅ 完了

すべての分析が完了しました！

### 生成されたファイル
- `ヒストグラム.png`
- `散布図.png`
- `相関分析.png`
- `回帰分析結果.txt`